# Pix2Pix Network, An Image-To-Image Translation Using Conditional GANs (CGANs)

Download the dataset from the following link 
http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/cityscapes.tar.gz

Import the modules.

In [1]:
import os
import sys
import random
import numpy as np
from PIL import Image
import keras.backend as K
from copy import deepcopy
from random import randint
from keras import initializers
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.layers.core import Activation
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD, Nadam,Adamax
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import  Flatten, Dropout, Lambda, Concatenate
from keras.layers import Dense, Reshape, Input, BatchNormalization, Concatenate
from keras.layers.convolutional import UpSampling2D, Convolution2D, MaxPooling2D,Deconvolution2D

Using TensorFlow backend.


The generator uses the U-NET architecture.

In [2]:
class Generator(object):
    def __init__(self, width = 28, height= 28, channels = 1):
        
        self.W = width
        self.H = height
        self.C = channels
        self.SHAPE = (width,height,channels)

        self.Generator = self.model()
        self.OPTIMIZER = Adam(lr=2e-4, beta_1=0.5,decay=1e-5)
        self.Generator.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER,metrics=['accuracy'])

        self.save_model()
        self.summary()

    def model(self):
        input_layer = Input(shape=self.SHAPE)
        
        down_1 = Convolution2D(64  , kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(input_layer)

        down_2 = Convolution2D(64*2, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(down_1)
        norm_2 = BatchNormalization()(down_2)

        down_3 = Convolution2D(64*4, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_2)
        norm_3 = BatchNormalization()(down_3)

        down_4 = Convolution2D(64*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_3)
        norm_4 = BatchNormalization()(down_4)

        down_5 = Convolution2D(64*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_4)
        norm_5 = BatchNormalization()(down_5)

        down_6 = Convolution2D(64*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_5)
        norm_6 = BatchNormalization()(down_6)

        down_7 = Convolution2D(64*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(norm_6)
        norm_7 = BatchNormalization()(down_7)


        upsample_1 = UpSampling2D(size=2)(norm_7)
        up_conv_1 = Convolution2D(64*8, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_1)
        norm_up_1 = BatchNormalization(momentum=0.8)(up_conv_1)
        add_skip_1 = Concatenate()([norm_up_1,norm_6])


        upsample_2 = UpSampling2D(size=2)(add_skip_1)
        up_conv_2 = Convolution2D(64*8, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_2)
        norm_up_2 = BatchNormalization(momentum=0.8)(up_conv_2)
        add_skip_2 = Concatenate()([norm_up_2,norm_5])

        upsample_3 = UpSampling2D(size=2)(add_skip_2)
        up_conv_3 = Convolution2D(64*8, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_3)
        norm_up_3 = BatchNormalization(momentum=0.8)(up_conv_3)
        add_skip_3 = Concatenate()([norm_up_3,norm_4])


        upsample_4 = UpSampling2D(size=2)(add_skip_3)
        up_conv_4 = Convolution2D(64*4, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_4)
        norm_up_4 = BatchNormalization(momentum=0.8)(up_conv_4)
        add_skip_4 = Concatenate()([norm_up_4,norm_3])

        upsample_5 = UpSampling2D(size=2)(add_skip_4)
        up_conv_5 = Convolution2D(64*2, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_5)
        norm_up_5 = BatchNormalization(momentum=0.8)(up_conv_5)
        add_skip_5 = Concatenate()([norm_up_5,norm_2])

        upsample_6 = UpSampling2D(size=2)(add_skip_5)
        up_conv_6 = Convolution2D(64, kernel_size=4, strides=1, padding='same',activation='relu')(upsample_6)
        norm_up_6 = BatchNormalization(momentum=0.8)(up_conv_6)
        add_skip_6 = Concatenate()([norm_up_6,down_1])

        last_upsample = UpSampling2D(size=2)(add_skip_6)
        output_layer = Convolution2D(self.C, kernel_size=4, strides=1, padding='same',activation='tanh')(last_upsample)
        
        return Model(input_layer,output_layer)

    def summary(self):
        return self.Generator.summary()

    def save_model(self):
        plot_model(self.Generator, to_file='./data/out/Generator_Model.png')

Discriminator network.

In [3]:
class Discriminator(object):
    def __init__(self, width = 28, height= 28, channels = 1, starting_filters=64):
        self.W = width
        self.H = height
        self.C = channels
        self.CAPACITY = width*height*channels
        self.SHAPE = (width,height,channels)
        self.FS = starting_filters #FilterStart
        
        self.Discriminator = self.model()
        self.OPTIMIZER = Adam(lr=2e-4, beta_1=0.5,decay=1e-5)
        self.Discriminator.compile(loss='mse', optimizer=self.OPTIMIZER, metrics=['accuracy'] )

        self.save_model()
        self.summary()

    def model(self):


        input_A = Input(shape=self.SHAPE)
        input_B = Input(shape=self.SHAPE)
        input_layer = Concatenate(axis=-1)([input_A, input_B])

        up_layer_1 = Convolution2D(self.FS, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(input_layer)

        up_layer_2 = Convolution2D(self.FS*2, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(up_layer_1)
        leaky_layer_2 =  BatchNormalization(momentum=0.8)(up_layer_2)

        up_layer_3 = Convolution2D(self.FS*4, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(leaky_layer_2)
        leaky_layer_3 =  BatchNormalization(momentum=0.8)(up_layer_3)

        up_layer_4 = Convolution2D(self.FS*8, kernel_size=4, strides=2, padding='same',activation=LeakyReLU(alpha=0.2))(leaky_layer_3)
        leaky_layer_4 = BatchNormalization(momentum=0.8)(up_layer_4)

        output_layer = Convolution2D(1, kernel_size=4, strides=1, padding='same')(leaky_layer_4)
        
        return Model([input_A, input_B],output_layer)

    def summary(self):
        return self.Discriminator.summary()

    def save_model(self):
        plot_model(self.Discriminator, to_file='./data/out/Discriminator_Model.png')

GAN Network

In [4]:
class GAN(object):
    def __init__(self, model_inputs=[],model_outputs=[]):
        self.OPTIMIZER = SGD(lr=2e-4,nesterov=True)

        self.inputs = model_inputs
        self.outputs = model_outputs
        self.gan_model = Model(inputs = self.inputs, outputs = self.outputs)
        self.OPTIMIZER = Adam(lr=2e-4, beta_1=0.5)
        self.gan_model.compile(loss=['mse', 'mae'],
                            loss_weights=[  1, 100],
                            optimizer=self.OPTIMIZER)
        self.save_model()
        self.summary()

    def model(self):
        model = Model()
        return model

    def summary(self):
        return self.gan_model.summary()

    def save_model(self):
        plot_model(self.gan_model, to_file='./data/out/GAN_Model.png')

Trainer class for training

In [5]:
class Trainer:
    def __init__(self, height = 256, width = 256, channels=3, epochs = 50000, batch = 1, checkpoint = 50, train_data_path = '',test_data_path=''):
        self.EPOCHS = epochs
        self.BATCH = batch
        self.H = height
        self.W = width
        self.C = channels
        self.CHECKPOINT = checkpoint

        self.X_train_B, self.X_train_A = self.load_data(train_data_path)
        self.X_test_B, self.X_test_A  = self.load_data(test_data_path)


        self.generator = Generator(height=self.H, width=self.W, channels=self.C)

        self.orig_A = Input(shape=(self.W, self.H, self.C))
        self.orig_B = Input(shape=(self.W, self.H, self.C))

        self.fake_A = self.generator.Generator(self.orig_B)

        self.discriminator = Discriminator(height=self.H, width=self.W, channels=self.C)
        self.discriminator.trainable = False
        self.valid = self.discriminator.Discriminator([self.fake_A,self.orig_B])

        model_inputs  = [self.orig_A,self.orig_B]
        model_outputs = [self.valid, self.fake_A]
        self.gan = GAN(model_inputs=model_inputs,model_outputs=model_outputs)
        
        

    def load_data(self,data_path):
        listOFFiles = self.grabListOfFiles(data_path,extension="jpg")
        imgs_temp = np.array(self.grabArrayOfImages(listOFFiles))
        imgs_A = []
        imgs_B = []
        for img in imgs_temp:
            imgs_A.append(img[:,:self.H])
            imgs_B.append(img[:,self.H:])

        imgs_A_out = self.norm_and_expand(np.array(imgs_A))
        imgs_B_out = self.norm_and_expand(np.array(imgs_B))

        return imgs_A_out, imgs_B_out

    def norm_and_expand(self,arr):
        arr = (arr.astype(np.float32) - 127.5)/127.5
        normed = np.expand_dims(arr, axis=3)
        return normed

    def grabListOfFiles(self,startingDirectory,extension=".webp"):
        listOfFiles = []
        for file in os.listdir(startingDirectory):
            if file.endswith(extension):
                listOfFiles.append(os.path.join(startingDirectory, file))
        return listOfFiles

    def grabArrayOfImages(self,listOfFiles,gray=False):
        imageArr = []
        for f in listOfFiles:
            if gray:
                im = Image.open(f).convert("L")
            else:
                im = Image.open(f).convert("RGB")
            imData = np.asarray(im)
            imageArr.append(imData)
        return imageArr


    def train(self):
        for e in range(self.EPOCHS):
            b = 0
            X_train_A_temp = deepcopy(self.X_train_A)
            X_train_B_temp = deepcopy(self.X_train_B)

            number_of_batches = len(self.X_train_A)
        
            for b in range(number_of_batches):
                # Train Discriminator
                # Grab Real Images for this training batch
                starting_ind = randint(0, (len(X_train_A_temp)-1))
                real_images_raw_A = X_train_A_temp[ starting_ind : (starting_ind + 1) ]
                real_images_raw_B = X_train_B_temp[ starting_ind : (starting_ind + 1) ]

                # Delete the images used until we have none left
                X_train_A_temp = np.delete(X_train_A_temp,range(starting_ind,(starting_ind + 1)),0)
                X_train_B_temp = np.delete(X_train_B_temp,range(starting_ind,(starting_ind + 1)),0)

                batch_A = real_images_raw_A.reshape( 1, self.W, self.H, self.C )
                batch_B = real_images_raw_B.reshape( 1, self.W, self.H, self.C )

                # PatchGAN
                y_valid = np.ones((1,)+(int(self.W / 2**4), int(self.W / 2**4), 1))
                y_fake = np.zeros((1,)+(int(self.W / 2**4), int(self.W / 2**4), 1))

                fake_A = self.generator.Generator.predict(batch_B)

                # Now, train the discriminator with this batch of reals
                discriminator_loss_real = self.discriminator.Discriminator.train_on_batch([batch_A,batch_B],y_valid)[0]
                discriminator_loss_fake = self.discriminator.Discriminator.train_on_batch([fake_A,batch_B],y_fake)[0]
                full_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

                generator_loss = self.gan.gan_model.train_on_batch([batch_A, batch_B],[y_valid,batch_A])    

                print ('Batch: '+str(int(b))+', [Full Discriminator :: Loss: '+str(full_loss)+'], [ Generator :: Loss: '+str(generator_loss)+']')
                if b % self.CHECKPOINT == 0 :
                    label = str(e)+'_'+str(b)
                    self.plot_checkpoint(label)

            print ('Epoch: '+str(int(e))+', [Full Discriminator :: Loss:'+str(full_loss)+'], [ Generator :: Loss: '+str(generator_loss)+']')
                        
        return

    def plot_checkpoint(self,b):
        orig_filename = "./data/out/batch_check_"+str(b)+"_original.png"

        r, c = 3, 3
        random_inds = random.sample(range(len(self.X_test_A)),3)
        imgs_A = self.X_test_A[random_inds].reshape(3, self.W, self.H, self.C )
        imgs_B = self.X_test_B[random_inds].reshape( 3, self.W, self.H, self.C )
        fake_A = self.generator.Generator.predict(imgs_B)

        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Style', 'Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("./data/out/batch_check_"+str(b)+".png")
        plt.close('all')

        return

In [6]:
#discriminator = Discriminator(256, 256, 3)
#generator = Generator(256, 256, 3)

Create instance of Trainer, with EPOCHS value 1 and BATCH of 1

In [7]:
HEIGHT  = 256
WIDTH   = 256
CHANNELS = 3
#EPOCHS = 5
EPOCHS = 1
BATCH = 1
CHECKPOINT = 100
TRAIN_PATH = "./data/cityscapes/train/"
TEST_PATH = "./data/cityscapes/val/"

trainer = Trainer(height=HEIGHT,width=WIDTH, channels=CHANNELS,epochs =EPOCHS,\
                 batch=BATCH,\
                 checkpoint=CHECKPOINT,\
                 train_data_path=TRAIN_PATH,\
                 test_data_path=TEST_PATH)


C:\Users\maninaya\Anaconda3\envs\env\lib\site-packages\keras\activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 3136        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 128)  131200      conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 128)  512         conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 256, 256, 6)  0           input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 128, 128, 64) 6208        concatenate_7[0][0]              
__________

Start training

In [8]:
trainer.train()

Batch: 0, [Full Discriminator :: Loss: 6.698875427246094], [ Generator :: Loss: [99.54007, 30.196295, 0.6934377]]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Batch: 1, [Full Discriminator :: Loss: 8.405410766601562], [ Generator :: Loss: [77.71934, 3.0983768, 0.7462096]]
Batch: 2, [Full Discriminator :: Loss: 7.120274543762207], [ Generator :: Loss: [71.098434, 3.7753778, 0.6732306]]
Batch: 3, [Full Discriminator :: Loss: 3.363682508468628], [ Generator :: Loss: [74.306015, 9.074961, 0.65231055]]
Batch: 4, [Full Discriminator :: Loss: 2.8947370052337646], [ Generator :: Loss: [76.20352, 6.381177, 0.6982234]]
Batch: 5, [Full Discriminator :: Loss: 2.1841342449188232], [ Generator :: Loss: [64.48522, 1.5167869, 0.6296843]]
Batch: 6, [Full Discriminator :: Loss: 1.7019485235214233], [ Generator :: Loss: [52.663326, 1.4109839, 0.5125234]]
Batch: 7, [Full Discriminator :: Loss: 1.044288158416748], [ Generator :: Loss: [61.547966, 1.4654245, 0.6008254]]
Batch: 8, [Full Discriminator :: Loss: 1.039643406867981], [ Generator :: Loss: [55.49375, 0.9809724, 0.5451278]]
Batch: 9, [Full Discriminator :: Loss: 0.9713795185089111], [ Generator :: Loss: [

Batch: 71, [Full Discriminator :: Loss: 0.45747289061546326], [ Generator :: Loss: [25.685972, 0.2898104, 0.25396162]]
Batch: 72, [Full Discriminator :: Loss: 0.446033239364624], [ Generator :: Loss: [30.419205, 0.26511073, 0.30154094]]
Batch: 73, [Full Discriminator :: Loss: 0.49205100536346436], [ Generator :: Loss: [21.369713, 0.33846188, 0.21031252]]
Batch: 74, [Full Discriminator :: Loss: 0.4570978283882141], [ Generator :: Loss: [28.541979, 0.27582586, 0.28266153]]
Batch: 75, [Full Discriminator :: Loss: 0.3697962164878845], [ Generator :: Loss: [28.658756, 0.24993767, 0.2840882]]
Batch: 76, [Full Discriminator :: Loss: 0.4648888409137726], [ Generator :: Loss: [32.637543, 0.36853433, 0.32269007]]
Batch: 77, [Full Discriminator :: Loss: 0.3398706912994385], [ Generator :: Loss: [26.356455, 0.29468566, 0.2606177]]
Batch: 78, [Full Discriminator :: Loss: 0.3324142098426819], [ Generator :: Loss: [30.272915, 0.3899527, 0.29882962]]
Batch: 79, [Full Discriminator :: Loss: 0.378000676

Batch: 140, [Full Discriminator :: Loss: 0.3627251982688904], [ Generator :: Loss: [23.821825, 0.3691479, 0.23452677]]
Batch: 141, [Full Discriminator :: Loss: 0.3139342963695526], [ Generator :: Loss: [24.725521, 0.21232072, 0.24513201]]
Batch: 142, [Full Discriminator :: Loss: 0.26222649216651917], [ Generator :: Loss: [23.522924, 0.25917903, 0.23263747]]
Batch: 143, [Full Discriminator :: Loss: 0.2763967514038086], [ Generator :: Loss: [18.681091, 0.20715028, 0.18473941]]
Batch: 144, [Full Discriminator :: Loss: 0.3148163855075836], [ Generator :: Loss: [16.430542, 0.16566783, 0.16264874]]
Batch: 145, [Full Discriminator :: Loss: 0.2939367890357971], [ Generator :: Loss: [21.305756, 0.15970588, 0.21146049]]
Batch: 146, [Full Discriminator :: Loss: 0.27908170223236084], [ Generator :: Loss: [24.071938, 0.25267065, 0.23819268]]
Batch: 147, [Full Discriminator :: Loss: 0.274921178817749], [ Generator :: Loss: [24.663866, 0.24866937, 0.24415196]]
Batch: 148, [Full Discriminator :: Loss:

...

Batch: 2641, [Full Discriminator :: Loss: 0.4576927721500397], [ Generator :: Loss: [17.522219, 0.10369733, 0.17418522]]
Batch: 2642, [Full Discriminator :: Loss: 0.06868717074394226], [ Generator :: Loss: [23.451664, 0.21892716, 0.23232736]]
Batch: 2643, [Full Discriminator :: Loss: 0.06540898233652115], [ Generator :: Loss: [14.877005, 0.11629897, 0.14760706]]
Batch: 2644, [Full Discriminator :: Loss: 0.14257556200027466], [ Generator :: Loss: [17.5777, 0.066553615, 0.17511146]]
Batch: 2645, [Full Discriminator :: Loss: 0.06102500110864639], [ Generator :: Loss: [13.48154, 0.15211749, 0.13329422]]
Batch: 2646, [Full Discriminator :: Loss: 0.05183672904968262], [ Generator :: Loss: [18.545544, 0.13576387, 0.1840978]]
Batch: 2647, [Full Discriminator :: Loss: 0.24984559416770935], [ Generator :: Loss: [15.932533, 0.2278251, 0.15704708]]
Batch: 2648, [Full Discriminator :: Loss: 0.04302317649126053], [ Generator :: Loss: [15.806676, 0.2743525, 0.15532324]]
Batch: 2649, [Full Discriminat

Batch: 2709, [Full Discriminator :: Loss: 0.032362665981054306], [ Generator :: Loss: [13.245188, 0.18080358, 0.13064384]]
Batch: 2710, [Full Discriminator :: Loss: 0.12441787868738174], [ Generator :: Loss: [17.794613, 0.2706489, 0.17523964]]
Batch: 2711, [Full Discriminator :: Loss: 0.16982628405094147], [ Generator :: Loss: [18.16215, 0.24289626, 0.17919253]]
Batch: 2712, [Full Discriminator :: Loss: 0.04343441501259804], [ Generator :: Loss: [19.480589, 0.23165019, 0.1924894]]
Batch: 2713, [Full Discriminator :: Loss: 0.14063769578933716], [ Generator :: Loss: [13.665252, 0.13371556, 0.13531536]]
Batch: 2714, [Full Discriminator :: Loss: 0.18566010892391205], [ Generator :: Loss: [15.51018, 0.1999831, 0.15310198]]
Batch: 2715, [Full Discriminator :: Loss: 0.11121240258216858], [ Generator :: Loss: [22.435904, 0.31986916, 0.22116034]]
Batch: 2716, [Full Discriminator :: Loss: 0.08045892417430878], [ Generator :: Loss: [17.48868, 0.16289727, 0.17325783]]
Batch: 2717, [Full Discrimina

Batch: 2777, [Full Discriminator :: Loss: 0.08582764118909836], [ Generator :: Loss: [20.107634, 0.20428964, 0.19903344]]
Batch: 2778, [Full Discriminator :: Loss: 0.15416136384010315], [ Generator :: Loss: [19.535364, 0.29280356, 0.19242561]]
Batch: 2779, [Full Discriminator :: Loss: 0.08756875991821289], [ Generator :: Loss: [16.335869, 0.17990611, 0.16155961]]
Batch: 2780, [Full Discriminator :: Loss: 0.0750306248664856], [ Generator :: Loss: [10.465619, 0.047021143, 0.10418598]]
Batch: 2781, [Full Discriminator :: Loss: 0.1185227781534195], [ Generator :: Loss: [21.257402, 0.095937856, 0.21161464]]
Batch: 2782, [Full Discriminator :: Loss: 0.17655028402805328], [ Generator :: Loss: [16.72789, 0.09159576, 0.16636293]]
Batch: 2783, [Full Discriminator :: Loss: 0.07801707834005356], [ Generator :: Loss: [10.811894, 0.10694827, 0.107049465]]
Batch: 2784, [Full Discriminator :: Loss: 0.19910117983818054], [ Generator :: Loss: [21.272867, 0.3430019, 0.20929864]]
Batch: 2785, [Full Discri

Batch: 2845, [Full Discriminator :: Loss: 0.10375946760177612], [ Generator :: Loss: [13.266759, 0.19485827, 0.130719]]
Batch: 2846, [Full Discriminator :: Loss: 0.23330524563789368], [ Generator :: Loss: [16.559637, 0.24774206, 0.16311896]]
Batch: 2847, [Full Discriminator :: Loss: 0.2943092882633209], [ Generator :: Loss: [15.75422, 0.21007612, 0.15544143]]
Batch: 2848, [Full Discriminator :: Loss: 0.26560187339782715], [ Generator :: Loss: [12.308502, 0.17522864, 0.12133273]]
Batch: 2849, [Full Discriminator :: Loss: 0.2548394501209259], [ Generator :: Loss: [20.54636, 0.27708703, 0.20269273]]
Batch: 2850, [Full Discriminator :: Loss: 0.14673160016536713], [ Generator :: Loss: [21.244164, 0.4391016, 0.20805061]]
Batch: 2851, [Full Discriminator :: Loss: 0.2304437905550003], [ Generator :: Loss: [15.8057785, 0.16212702, 0.15643652]]
Batch: 2852, [Full Discriminator :: Loss: 0.4570194482803345], [ Generator :: Loss: [17.117064, 0.23189345, 0.1688517]]
Batch: 2853, [Full Discriminator 

Batch: 2913, [Full Discriminator :: Loss: 0.19775769114494324], [ Generator :: Loss: [12.009283, 0.16786304, 0.1184142]]
Batch: 2914, [Full Discriminator :: Loss: 0.04898422956466675], [ Generator :: Loss: [18.562763, 0.16153969, 0.18401225]]
Batch: 2915, [Full Discriminator :: Loss: 0.06430866569280624], [ Generator :: Loss: [14.695024, 0.12654424, 0.1456848]]
Batch: 2916, [Full Discriminator :: Loss: 0.09499618411064148], [ Generator :: Loss: [17.77872, 0.12298897, 0.17655732]]
Batch: 2917, [Full Discriminator :: Loss: 0.0728980228304863], [ Generator :: Loss: [21.022934, 0.09027437, 0.2093266]]
Batch: 2918, [Full Discriminator :: Loss: 0.11122873425483704], [ Generator :: Loss: [17.886103, 0.1058197, 0.17780283]]
Batch: 2919, [Full Discriminator :: Loss: 0.09249135851860046], [ Generator :: Loss: [16.713608, 0.23876995, 0.16474839]]
Batch: 2920, [Full Discriminator :: Loss: 0.12838932871818542], [ Generator :: Loss: [18.193668, 0.067927875, 0.1812574]]
Batch: 2921, [Full Discriminat